In [0]:
dbutils.fs.ls('/Volumes/workspace/default/tmp_customer/deliveries.csv')

In [0]:
ipl_df = spark.read.csv('/Volumes/workspace/default/tmp_customer/deliveries.csv', header=True)

display(ipl_df)
ipl_df.printSchema()

In [0]:
ipl_df.printSchema()

In [0]:
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.sql.types import StructField, StringType, IntegerType, StructType,FloatType



In [0]:

int_col = ['match_id' , 'inning','over','ball','batsman_run','extra_runs','total_runs', 'is_wicket']

fields = [StructField(col, IntegerType(), nullable=True) if col in int_col else StructField(col, StringType(), nullable=True) for col in ipl_df.columns]



In [0]:

fields

In [0]:

new_df= spark.read.option('header', 'true').schema(StructType(fields)).csv('/Volumes/workspace/default/tmp_customer/deliveries.csv')

new_df.display()

In [0]:
new_df.select('match_id').distinct().sort(F.col('match_id').desc()).show()

In [0]:

new_df.printSchema()

In [0]:
df = new_df.filter(F.col('match_id') == 1426312)
df.display()

In [0]:
first_innings_batting = df.filter(df.inning ==1)

first_innings_batting.display()

In [0]:
scorecard_df = first_innings_batting.filter('extras_type is null').groupBy('batter').agg(F.sum('batsman_runs').alias('runs'),
                                                           F.count('ball').alias('balls'),
                                                           F.count(F.when(first_innings_batting.batsman_runs == 4,1)).alias('4s'),
                                                           F.count(F.when(first_innings_batting.batsman_runs == 6,1)).alias('6s'),
                                                           F.round(F.sum('batsman_runs')*100/F.count('ball'),2).alias('S/R'))



In [0]:
scorecard_df.display()

In [0]:
scorecard_df.filter("batter =='AK Markram'").display()

In [0]:
first_innings_batting.filter(F.col('batter')=='AK Markram').display()


In [0]:
batsman_order = (first_innings_batting.withColumn('over-ball', (F.concat(F.col('over'),F.lit("."),F.col("ball"))).cast(FloatType())).groupBy("batter").agg(F.min("over-ball").alias("order")).orderBy("order"))

In [0]:
batsman_order.display()

In [0]:
from pyspark.sql.window import Window

batting_order_df = batsman_order.withColumn("batting_order", F.row_number().over(Window.orderBy("order")))

batting_order_df.display()

In [0]:
scorecard_df.join(batting_order_df, on=['batter','batter'], how='inner').display()

In [0]:
scorecard_bowler_df = (first_innings_batting.groupBy('bowler').agg(F.sum('total_runs').alias('run_conceded'),
                                                                  F.sum((F.when((F.col('extras_type')== "legbyes") 
                                                                               | (F.col("extras_type")== "byes"), F.col('extra_runs')))).alias('not_by_bowler'),
                                        F.count(F.when((F.col('extras_type').isNull()) | (F.col('extras_type') == "legbyes") | (F.col("extras_type") == "byes"),1)).alias('balls'), 
                                                                        F.count(F.when(F.col('is_wicket')==1,1)).alias('W'))
)

In [0]:
scorecard_bowler_df.display()

In [0]:
scorecard_bowler_df.select(F.col('bowler'), 
                           (F.col('balls')/6).alias('o'),
                           (F.col('run_conceded') -F.col('not_by_bowler')).alias('R'),
                           F.col('W'),
                           (F.col('run_conceded')/(F.col('balls')/6)).alias('Econ')).display()


In [0]:
scorecard_bowler_df = scorecard_bowler_df.select(F.col('bowler'),
                                                 F.concat(F.floor(F.col('balls')/6), F.lit("."), (F.col('balls')%6)).alias('O'),
                                                 (F.col('run_conceded') - F.coalesce(F.col('not_by_bowler'),F.lit(0))).alias('R'),
                                                 F.col('W'),
                                                 F.round((F.col('run_conceded')/(F.col('balls')/6)),2).alias('Econ'))


scorecard_bowler_df.display()

In [0]:
#Maiden Over Calculation

maiden_bowler_df = first_innings_batting.groupBy('bowler', 'over').agg(F.sum('total_runs').alias('run_conceded'),
                                                                       F.count(F.col('over')).alias('balls'),
                                                                       F.sum((F.when((F.col('extras_type')== "legbyes") 
                                                                               | (F.col("extras_type")== "byes"), F.col('extra_runs')))).alias('not_by_bowler'))

In [0]:
maiden_bowler_df.display()

In [0]:
maiden_bowler_df =  (maiden_bowler_df.withColumn('runs_by_bowler', F.col('run_conceded') - F.coalesce(F.col('not_by_bowler'), F.lit(0))))


In [0]:
maiden_bowler_df = maiden_bowler_df.withColumn('runs_by_bowler', F.col('run_conceded') - F.coalesce(F.col('not_by_bowler'), F.lit(0)))\
    .filter((F.col('runs_by_bowler')==0) & (F.col('balls')== 6))


maiden_bowler_df.display()

In [0]:
scorecard_bowler_df.join(maiden_bowler_df, on=['bowler','bowler'], how='left').fillna(value=0).display()